In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from Lab_Analyses.Utilities.get_dFoF import get_dFoF
%matplotlib notebook
sns.set()
sns.set_style("ticks")

In [2]:
initial_path = r"D:\Suite2P_data\paAIP2"
mouse_ids = ["JW157", "JW158", "JW159"]

periods = ["Early", "Middle", "Late"]

In [4]:
import os
ops_path = os.path.join(initial_path, mouse_ids[0], periods[0], "JW157_230227_ops.npy")
ops = np.load(ops_path, allow_pickle=True)

In [7]:
ops.item()

{'suite2p_version': '0.12.0',
 'look_one_level_down': 0.0,
 'fast_disk': 'D:/Suite2P_data/paAIP2/JW157/Early',
 'delete_bin': True,
 'mesoscan': False,
 'bruker': False,
 'bruker_bidirectional': False,
 'h5py': [],
 'h5py_key': 'data',
 'nwb_file': '',
 'nwb_driver': '',
 'nwb_series': '',
 'save_path0': 'D:/Suite2P_data/paAIP2/JW157/Early',
 'save_folder': 'suite2p',
 'subfolders': [],
 'move_bin': False,
 'nplanes': 1,
 'nchannels': 1,
 'functional_chan': 1,
 'tau': 1.25,
 'fs': 30.0,
 'force_sktiff': False,
 'frames_include': -1,
 'multiplane_parallel': 0.0,
 'ignore_flyback': [],
 'preclassify': 0.0,
 'save_mat': True,
 'save_NWB': 0.0,
 'combined': 1.0,
 'aspect': 1.0,
 'do_bidiphase': False,
 'bidiphase': 0.0,
 'bidi_corrected': False,
 'do_registration': 0,
 'two_step_registration': 0.0,
 'keep_movie_raw': False,
 'nimg_init': 300,
 'batch_size': 500,
 'maxregshift': 0.1,
 'align_by_chan': 1,
 'reg_tif': False,
 'reg_tif_chan2': False,
 'subpixel': 10,
 'smooth_sigma_time': 0.0,

In [ ]:
import os
F = {}
iscell = {}
ops = {}
for mouse in mouse_ids:
    mouse_f = {}
    mouse_cell = {}
    mouse_ops = {}
    for period in periods:
        load_path = os.path.join(initial_path, mouse, period, "suite2p", "plane0")
        f_name = os.path.join(load_path, "F.npy")
        cell_name = os.path.join(load_path, "iscell.npy")
        ops_name = os.path.join(load_path, "stat.npy")
        f = np.load(f_name)
        cell = np.load(cell_name)[:, 0].astype(int)
        cell = cell.astype(bool)
        ops_n = np.load(ops_name, allow_pickle=True)
        mouse_f[period] = f
        mouse_cell[period] = cell
        mouse_ops[period] = ops_n
    F[mouse] = mouse_f
    iscell[mouse] = mouse_cell
    ops[mouse] = mouse_ops

In [ ]:
a = ops["JW157"]["Early"]
a[iscell["JW157"]["Early"]][0]["med"]

In [ ]:
fluorescence = {}
for key, value in F.items():
    new_f = {}
    for period, data in value.items():
        cells = iscell[key][period]
        f_cells = data[cells]
        new_f[period] = f_cells.T
    fluorescence[key] = new_f
        

In [ ]:
dFoF = {}
processed_dFoF = {}
for key, value in fluorescence.items():
    new_d = {}
    new_pd = {}
    for period, data in value.items():
        d = np.zeros(data.shape)
        pd = np.zeros(data.shape)
        for i in range(data.shape[1]):
            x, y, _ = get_dFoF(data[:, i], 30, 0.5, None, None)
            d[:, i] = x
            pd[:, i] = y
        new_d[period] = d
        new_pd[period] = pd
    dFoF[key] = new_d
    processed_dFoF[key] = new_pd
    

In [ ]:
counts = np.zeros((3,4))
counts

In [ ]:
for col, (key, value) in enumerate(dFoF.items()):
    for row, (k, v) in enumerate(value.items()):
        counts[row, col] = v.shape[1]

In [ ]:
count_dict = {"paAIP2": counts}

In [ ]:
from Lab_Analyses.Plotting.plot_multi_line_plot import plot_multi_line_plot

In [ ]:

plot_multi_line_plot(
    count_dict,
    x_vals = [0, 1, 2],
    plot_ind=True,
    figsize=(4,3),
    title=None,
    ytitle="Number of detected cells",
    xtitle="Imaging session",
    ylim=(0,300),
    line_color="mediumblue",
    face_color="white",
    m_size=7,
    linewidth=1.5,
    linestyle="-",
    axis_width=1.5,
    minor_ticks="y",
    tick_len=3,
    ax=None,
    legend=False,
    save=False,
    save_path=None
)

In [ ]:
plot_data = processed_dFoF["JW156"]["Late"]

In [ ]:
%matplotlib qt
fig, ax = plt.subplots(figsize=(8,10))

for i in range(50):
    x = np.linspace(0, len(plot_data[5000:50000, i]) / 30, len(plot_data[5000:50000, i]))
    ax.plot(x, plot_data[5000:50000, i] + i, linewidth=0.5)

plt.xlabel("Time (s)")
plt.ylabel("Cell number")
plt.xlim(0, 1000)
plt.tick_params(left=False) 
ax.set_yticklabels([])
sns.despine(top=True, right=True, left=True, bottom=False)

In [ ]:
from Lab_Analyses.Population_Analysis.preprocess_population_data import organize_population_data

In [ ]:
organize_population_data(
    mouse_id="JW157",
    roi_match=False,
    sensor="RCaMP2",
    zoom_factor=2,
    deconvolve=True,
    reprocess=True,
    save=True,
)

In [ ]:
import numpy as np
a = np.array([0,0,0,0,0,0,1,0,0])
np.nonzero((a[1:] > 0).astype(int) & (a[:-1] < 1).astype(int))[0]

In [ ]:
from Lab_Analyses.Utilities.save_load_pickle import load_pickle

In [ ]:
data = load_pickle([r"C:\Users\Jake\Desktop\Analyzed_data\individual\JW157\population_data\JW157_Early_population_data"])[0]

In [ ]:
spikes = data.estimated_spikes

In [ ]:
dFoF = data.processed_dFoF

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
plt.figure()
plt.plot(spikes[28000:30000, 0])
plt.figure()
plt.plot(dFoF[28000:30000, 0])

In [ ]:
from scipy.ndimage import uniform_filter1d

In [ ]:
binned_spikes = uniform_filter1d(spikes, 6, axis=0)

In [ ]:
plt.figure()
plt.plot(binned_spikes[28000:30000, 0])

In [ ]:
from skimage.measure import block_reduce
import numpy as np

ds_spikes = block_reduce(spikes[:, 0].reshape(-1,1), block_size=(6,1), func=np.mean)

In [ ]:
plt.figure()
plt.plot(ds_spikes)

In [ ]:
from Lab_Analyses.Utilities.event_detection import event_detection

In [ ]:
activity_dFoF, _, _ = event_detection(dFoF, threshold=3, lower_threshold=1, lower_limit=0, sampling_rate=30, filt_poly=3, sec_smooth=1)

In [ ]:
activity_spikes, _, _ = event_detection(binned_spikes, threshold=3, lower_threshold=1, lower_limit=0, sampling_rate=30, filt_poly=3, sec_smooth=1)

In [ ]:
from Lab_Analyses.Utilities import data_utilities as d_utils

norm_spikes = d_utils.peak_normalize_data(binned_spikes)
norm_dFoF = d_utils.peak_normalize_data(dFoF)

In [ ]:
z_spikes = d_utils.z_score(spikes)
norm_z = d_utils.peak_normalize_data(z_spikes)

In [ ]:
import scipy.signal as sysignal
plt.figure()
plt.plot(norm_spikes[:, 0])
plt.plot(norm_dFoF[:, 0])
plt.plot(sysignal.savgol_filter(norm_spikes[:, 0], 31, 3), color="black")
plt.plot(sysignal.savgol_filter(sysignal.savgol_filter(norm_spikes[:, 0], 31, 3), 15, 2), color="blue")
#plt.plot(norm_z[:, 0], color="red")

In [ ]:
a = ["JW156", "JW157"]
b = ["JW160", "JW161"]
c = a + b
c

In [ ]:
a = []

for i in a:
    print(i)

In [ ]:
len(a) / 5

In [ ]:
if not len(a):
    print("empty")

In [1]:
from Lab_Analyses.Utilities.save_load_pickle import load_pickle

In [2]:
fname = r"C:\Users\Jake\Desktop\Analyzed_data\individual\JW092\spine_data\FOV1_apical\JW092_FOV1_apical_Early_dual_spine_data"

In [3]:
data = load_pickle([fname])[0]

In [4]:
spine_activity = data.spine_GluSnFr_activity
spine_groupings = data.spine_groupings

In [5]:
flags = data.spine_flags

from Lab_Analyses.Spine_Analysis_v2.spine_utilities import find_present_spines
present = find_present_spines(flags)

In [6]:
positions = data.spine_positions
from Lab_Analyses.Spine_Analysis_v2.spine_utilities import find_nearby_spines

nearby_spine_idxs = find_nearby_spines(positions, flags, spine_groupings, None, 5)

[]
[2, 3, 6]
[1, 3, 4, 5, 7]
[1, 2, 4, 5, 6]
[2, 3, 5, 7, 8, 9]
[2, 3, 4, 7, 8, 9]
[1, 3]
[2, 4, 5, 8, 9]
[4, 5, 7, 9]
[4, 5, 7, 8]
[1, 2, 3, 4]
[0, 2, 3, 4]
[0, 1, 3, 4]
[0, 1, 2, 4]
[0, 1, 2, 3, 5, 6]
[4, 6, 7]
[4, 5, 7]
[5, 6, 8, 9, 11]
[7, 9, 10, 11]
[7, 8, 10, 11, 12, 26]
[8, 9, 11, 12, 13, 23, 26]
[7, 8, 9, 10, 12, 13, 26]
[9, 10, 11, 13, 23, 26]
[10, 11, 12, 14, 23, 24, 25, 26]
[13, 15, 16, 23, 24, 25]
[14, 16, 24, 25]
[14, 15, 17]
[16, 18]
[17, 19]
[18]
[21, 22]
[20, 22]
[20, 21]
[10, 12, 13, 14, 24, 25, 26]
[13, 14, 15, 23, 25]
[13, 14, 15, 23, 24]
[9, 10, 11, 12, 13, 23]
[2]
[2, 3, 4, 5, 6]
[0, 1, 3, 4, 6]
[1, 2, 4, 5, 6]
[1, 2, 3, 5, 6]
[1, 3, 4, 6, 7]
[1, 2, 3, 4, 5, 7]
[5, 6, 8]
[7, 9, 10, 11]
[8, 10, 11, 12, 14]
[8, 9, 11, 12, 14]
[8, 9, 10, 12, 13, 14]
[9, 10, 11, 13, 14, 15]
[11, 12, 14, 15, 16]
[9, 10, 11, 12, 13]
[12, 13, 16, 17]
[13, 15, 17]
[15, 16, 18]
[17]


In [11]:
print(nearby_spine_idxs)
print(spine_groupings)
len(nearby_spine_idxs[0])

[array([], dtype=int32), array([2, 3, 6]), array([1, 3, 4, 5, 7]), array([1, 2, 4, 5, 6]), array([2, 3, 5, 7, 8, 9]), array([2, 3, 4, 7, 8, 9]), array([1, 3]), array([2, 4, 5, 8, 9]), array([4, 5, 7, 9]), array([4, 5, 7, 8]), array([11, 12, 13, 14]), array([10, 12, 13, 14]), array([10, 11, 13, 14]), array([10, 11, 12, 14]), array([10, 11, 12, 13, 15, 16]), array([14, 16, 17]), array([14, 15, 17]), array([15, 16, 18, 19, 21]), array([17, 19, 20, 21]), array([17, 18, 20, 21, 22, 55]), array([18, 19, 21, 22, 23, 33, 55]), array([17, 18, 19, 20, 22, 23, 55]), array([19, 20, 21, 23, 33, 55]), array([20, 21, 22, 24, 33, 34, 35, 55]), array([23, 25, 26, 33, 34, 35]), array([24, 26, 34, 35]), array([24, 25, 27]), array([26, 28]), array([27, 29]), array([28]), array([31, 32]), array([30, 32]), array([30, 31]), array([20, 22, 23, 24, 34, 35, 55]), array([23, 24, 25, 33, 35]), array([23, 24, 25, 33, 34]), array([38]), array([38, 39, 40, 41, 42]), array([36, 37, 39, 40, 42]), array([37, 38, 40, 41

0

In [10]:
for spine in range(spine_activity.shape[1]):
    if (nearby_spine_idxs[spine] is None) or (len(nearby_spine_idxs[spine]) == 0):
        print(f"{spine} has no neighbors")


0 has no neighbors
56 has no neighbors
57 has no neighbors
58 has no neighbors
59 has no neighbors
60 has no neighbors
61 has no neighbors
62 has no neighbors
63 has no neighbors
64 has no neighbors
65 has no neighbors
66 has no neighbors
67 has no neighbors
68 has no neighbors
69 has no neighbors
70 has no neighbors
71 has no neighbors
72 has no neighbors
73 has no neighbors
74 has no neighbors
75 has no neighbors
76 has no neighbors
77 has no neighbors
78 has no neighbors
79 has no neighbors
80 has no neighbors
81 has no neighbors
82 has no neighbors
83 has no neighbors
84 has no neighbors
85 has no neighbors
86 has no neighbors


In [16]:
import numpy as np
a = np.zeros((100, 10))
a

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 

In [20]:
a = np.zeros(10)
b = np.zeros(10)
c  = np.vstack((a,b)).T
c

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [24]:
poly_dendrite_dFoF = data.poly_dendrite_calcium_processed_dFoF

In [42]:
a = np.array([1,2,np.nan,4])
b = np.array([2,2,np.nan,2])

1 - a

array([ 0., -1., nan, -3.])